In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [2]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
!pip install findspark

In [4]:
import findspark
findspark.init()

In [5]:
!pip3 install pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 19.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044159 sha256=8cd03a149bcd91cb3c8b4f33cebfde0d7332d302d6457c28b6412bb9933c3297
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [6]:
# Инициализируем
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import udf, explode, rank, col, max, sum, desc, countDistinct
import re
from typing import List
import pyspark.sql as sql

In [7]:
spark = SparkSession \
    .builder \
    .appName("L2_reports_with_apache_spark") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0") \
    .getOrCreate()
spark.version

'3.1.1'

In [9]:
# Загружаем данные
import os
program_path = '/content/programming-languages.csv' # список языков
posts_path = '/content/posts_sample.xml' # выборка данных

In [10]:
# Читаем данные из файла в формате xml и каждую строку обернем в тег row
posts = spark.read.format('xml').options(rowTag='row').load(posts_path)

In [11]:
# Читаем данные из файла csv
program = spark.read \
      .option("header", True) \
      .option("inferSchema", True) \
      .option("DateTimeFormat", 'M/d/y H:m') \
      .csv(program_path)

**Задание**: Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы.
Получившийся отчёт сохранить в формате Apache Parquet.

In [12]:
def get_tags(string_tag):
    if string_tag is None:
        return []
    reg = r'<(.+?)>' # создаем шаблон регулярного выражения
    tags_list = re.findall(reg, string_tag) # извлекаем все совпадения и возвращаем список тегов
    return tags_list

def get_year(date_and_time):
    return date_and_time.year # возвращаем год из даты

# Используем пользовательские функции с udf
get_tags_list_udf = udf(get_tags, ArrayType(StringType())) # принимаем строку и возвращаем список строк
get_year_udf = udf(get_year, IntegerType()) # возвращаем целое число

# Добавляем два новых столбца к dataFrame
posts_data_simplified = posts \
                    .withColumn("tags", get_tags_list_udf(posts["_Tags"])) \
                    .withColumn("year", get_year_udf(posts["_LastActivityDate"]))

posts_data_simplified = posts_data_simplified.select(col("tags"), col("year"), col("_ViewCount").alias("views"))
first_rows = posts_data_simplified.head(10) # сохраняем первые 10 строк из dataFrame
for i, row in enumerate(first_rows):
    print(i+1, row)


1 Row(tags=['c#', 'floating-point', 'type-conversion', 'double', 'decimal'], year=2019, views=42817)
2 Row(tags=['html', 'css', 'internet-explorer-7'], year=2019, views=18214)
3 Row(tags=[], year=2017, views=None)
4 Row(tags=['c#', '.net', 'datetime'], year=2019, views=555183)
5 Row(tags=['c#', 'datetime', 'time', 'datediff', 'relative-time-span'], year=2019, views=149445)
6 Row(tags=[], year=2018, views=None)
7 Row(tags=['html', 'browser', 'timezone', 'user-agent', 'timezone-offset'], year=2019, views=176405)
8 Row(tags=['.net', 'math'], year=2018, views=123231)
9 Row(tags=[], year=2010, views=None)
10 Row(tags=[], year=2010, views=None)


In [18]:
# Теперь отсортируем, чтобы узнать самые популярные языки

# разобъем tags на отдельные строки
posts_sorted = posts_data_simplified.select("year", explode("tags").alias("tag"), "views")

# Группируем по году последней активности и тегам
# для каждого года и тега вычисляем сумму просмотров (в пределах одного года)
posts_sorted = posts_sorted.groupBy("year", "tag").agg(sum("views").alias("total_views"))

# Сортируем по убыванию количества просмотров
posts_sorted = posts_sorted.orderBy("year", desc("total_views"))
posts_sorted.show(10)


+----+-------------+-----------+
|year|          tag|total_views|
+----+-------------+-----------+
|2008|           c#|      25401|
|2008|         .net|      24321|
|2008|     database|      19682|
|2008|        local|      19682|
|2008|         java|      11532|
|2008|  inheritance|      10971|
|2008|accessibility|       7700|
|2008|    variables|       7700|
|2008|        excel|       6540|
|2008|   automation|       6540|
+----+-------------+-----------+
only showing top 10 rows



In [28]:
# Отсортируем по популярности

# Определяем спецификацию Window (отсортирована по годам и в порядке убывания по total_views)
window = Window.partitionBy("year").orderBy(posts_sorted["total_views"].desc())

# Добавляем колонку rank в dataFrame
# (содержит ранг каждой строки внутри окна)
rank_df = posts_sorted.withColumn("rank", rank().over(window))

# Фильтруем ранг
result_df = rank_df.filter(rank_df["rank"] <= 5)

# Сортируем по году и делаем в порядке убывания по количеству просмотров
result_df = result_df.select("year", "tag", "total_views")
data_sorted_result = result_df.orderBy("year", desc("total_views"))

data_sorted_result.show()

# Записываем в формате Apache Parquet
data_sorted_result.write.parquet("data_posts_sorted_result.parquet")

+----+--------------------+-----------+
|year|                 tag|total_views|
+----+--------------------+-----------+
|2008|                  c#|      25401|
|2008|                .net|      24321|
|2008|            database|      19682|
|2008|               local|      19682|
|2008|                java|      11532|
|2009|                  c#|      73661|
|2009|                .net|      39167|
|2009|              python|      32219|
|2009|                 c++|      29381|
|2009|            winforms|      25670|
|2010|                  c#|     128597|
|2010|              arrays|      80868|
|2010|                java|      53333|
|2010|multidimensional-...|      51865|
|2010|              matlab|      51865|
|2011|                  c#|     238076|
|2011|                java|     121315|
|2011|                .net|     120734|
|2011|                 css|     119302|
|2011|             android|     107283|
+----+--------------------+-----------+
only showing top 20 rows



In [29]:
# Тут я столкнулась с проблемой когда перезапускала код
# AnalysisException: path file:/content/data_posts_sorted_result.parquet already exists.
# Оказывается можно удалять только пустые каталоги
import shutil

In [30]:
# Удалим каталог
path_dir = "data_posts_sorted_result.parquet"
shutil.rmtree(path_dir)